## prediccion datos (error datos ourworld indata)


#### imports

In [1]:
from arcgis.gis import GIS
gis = GIS("home")
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn import linear_model
from scipy.special import expit
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score,mean_squared_error
import seaborn as sns
%matplotlib inline
import math
import datetime
one_day = datetime.timedelta(days=1)
from arcgis.features import FeatureLayerCollection

/opt/conda/lib/python3.6/site-packages/arcgis/gis/__init__.py:389: UserWarning: You are logged on as rmartin_esri_colombia with an administrator role, proceed with caution.
  self.users.me.username)


#### Now you are ready to start!

In [2]:
def fetch():
    df0=pd.read_csv('https://covid.ourworldindata.org/data/ecdc/full_data.csv')
    df1=pd.read_csv('https://covid.ourworldindata.org/data/ecdc/locations.csv')[['location','population']]
    df=pd.merge(df0,df1,left_on='location',right_on='location').dropna()
    # df.loc[df.location=='Colombia']
    sites=df.location.unique()
    df['casesPmill']=df['total_cases']/(df['population']/1e6)
    df['date']=pd.to_datetime(df['date'])
    return df,sites
def func_logistic(t, a, b, c):
    return c / (1 + a * np.exp(-b*t))
def create_dataframe_with_forecast(df0,site):    
    df_data=df0[df0['location']==site]
    N=df_data['population'].max()
    df_data=df_data[df_data.total_cases!=0]
    df_data['days']=np.arange(len(df_data.index))+1
    df_data=df_data[['location','date','total_cases','days','casesPmill']]
#     dfc['casesPmill']=dfc['total_cases']/(dfc['population']/1e6)
    bounds = (0, [100000., 1000., 1000000000.])
    p0 = np.random.exponential(size=3)
    step=0
    x,y=np.array(df_data.days),np.array(df_data.casesPmill) 
    while step<2:          
        popt,cov = curve_fit(func_logistic, x, y, bounds=bounds, p0=p0, maxfev=1000000)
        y_predict=func_logistic(x,*popt)
        r2=r2_score(y, y_predict)
        if r2<0.85:
            step+=1
        else:
            break
    df_forecast=pd.DataFrame()
    for i in np.arange(1,200):
        Day=df_data['days'].min()+i
        Date=df_data['date'].min()+(one_day*i)
        if Date <= datetime.datetime(2020, 6, 30):
            clean_negatives=lambda a: math.floor(func_logistic(a,*popt)) if math.floor(func_logistic(a,*popt))>0 else 0
            daycasespm=clean_negatives(Day)
            daycases=round((N/1e6)*(daycasespm))          
            day_df=pd.DataFrame([[site,
                                    Date,
                                    daycases,
                                    Day,daycasespm]],columns=list(df_data))
            df_forecast=pd.concat([df_forecast,day_df])
        else:
            break
    df_data['type']='data'
    df_forecast['type']='forecast'
    df_data_forecast=pd.concat([df_data,df_forecast])
    if y[-1]<=y[-2]:
        df_data_forecast['growth']='Estabilizado'
    else:
        df_data_forecast['growth']='Incrementando'
    
#     df_data_forecast['R2score']=r2
    print(site+'==> processed with r2:'+str(r2)+' in '+str(step)+' step')
    return df_data_forecast#,r2
def createFinal(df0,sites):
    Finaldf3=pd.DataFrame()
    scores={}
    for s in sites:
        dfi=create_dataframe_with_forecast(df0,s)
#         step=0
#         if r2 <0.7 or step==3:
#             dfi,r2=create_dataframe_with_forecast(df0,s)
#         scores[s]=r2
        Finaldf3=pd.concat([Finaldf3, dfi])
    Final3=Finaldf3.reset_index().drop(columns='index')
    fivehours=datetime.timedelta(hours=5)
    Final3['date']=Final3['date']+fivehours
    return Final3#, scores
def update():
    df,sites=fetch()
    Final=createFinal(df,sites)
    CSVout='forecast_pop_{}.csv'.format(str(datetime.date.today()))
    Final.to_csv(CSVout)
    item = gis.content.get("41ef4712f1404be2b7ec9e6f7f6d53f4")
    collection = FeatureLayerCollection.fromitem(item)
#     update_dict0 = {"capabilities": "Create,Delete,Query,Update,Editing"}
#     collection.manager.update_definition(update_dict0)
    collection.manager.overwrite(CSVout)
update()

Afghanistan==> processed with r2:0.9992814226175999 in 0 step
Albania==> processed with r2:0.9939845770466683 in 0 step
Algeria==> processed with r2:0.9955827354458613 in 0 step
Andorra==> processed with r2:0.9958970626128456 in 0 step
Angola==> processed with r2:0.9716871166970698 in 0 step
Anguilla==> processed with r2:0.822048980348324 in 2 step
Antigua and Barbuda==> processed with r2:0.9858448180260939 in 0 step
Argentina==> processed with r2:0.9885365051256084 in 0 step
Armenia==> processed with r2:0.995605548624542 in 0 step
Aruba==> processed with r2:0.9879294968013692 in 0 step
Australia==> processed with r2:0.9969649593329885 in 0 step
Austria==> processed with r2:0.9949450107440326 in 0 step
Azerbaijan==> processed with r2:0.9815828764194828 in 0 step
Bahamas==> processed with r2:0.9942749164716163 in 0 step
Bahrain==> processed with r2:0.9981676382631723 in 0 step
Bangladesh==> processed with r2:0.9976271560727039 in 0 step
Barbados==> processed with r2:0.9810928429214274 i

Nicaragua==> processed with r2:0.634013847261534 in 2 step
Niger==> processed with r2:0.9640881254188858 in 0 step
Nigeria==> processed with r2:0.9985897718134105 in 0 step
Northern Mariana Islands==> processed with r2:0.9050219226383236 in 0 step
Norway==> processed with r2:-2.220446049250313e-14 in 2 step
Oman==> processed with r2:0.9943911440805228 in 0 step
Pakistan==> processed with r2:0.9995608841658944 in 0 step
Palestine==> processed with r2:0.9833555338938406 in 0 step
Panama==> processed with r2:0.9933918119590482 in 0 step
Papua New Guinea==> processed with r2:0.9490077566808074 in 0 step
Paraguay==> processed with r2:0.9772173133689122 in 0 step
Peru==> processed with r2:0.9983873007071766 in 0 step
Philippines==> processed with r2:0.9923026893787231 in 0 step
Poland==> processed with r2:0.9914164856903364 in 0 step
Portugal==> processed with r2:0.9926809291722355 in 0 step
Puerto Rico==> processed with r2:0.9882262833535412 in 0 step
Qatar==> processed with r2:0.9990271746

In [48]:
create_dataframe_with_forecast(df,'Colombia')

Colombia==> processed


,location,date,total_cases,days,casesPmill,type,growth
3102,Colombia,2020-03-07,1,1,0.019653,data,Incrementando
3103,Colombia,2020-03-10,3,2,0.058959,data,Incrementando
3104,Colombia,2020-03-12,9,3,0.176877,data,Incrementando
3105,Colombia,2020-03-14,16,4,0.314448,data,Incrementando
3106,Colombia,2020-03-15,34,5,0.668201,data,Incrementando
...,...,...,...,...,...,...,...
0,Colombia,2020-06-11,14502,97,285.000000,forecast,Incrementando
0,Colombia,2020-06-12,14553,98,286.000000,forecast,Incrementando
0,Colombia,2020-06-13,14603,99,287.000000,forecast,Incrementando
0,Colombia,2020-06-14,14603,100,287.000000,forecast,Incrementando
